In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

from tqdm.autonotebook import tqdm, trange

import torchvision
from torchvision import models, datasets, transforms
import os

sns.set(font_scale=1.4, style='whitegrid')

In [15]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}
data_dir = './hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=3) for x in ['train', 'val']}

dataset_size = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_name = image_datasets['train'].classes

In [3]:
model_inception_extractor = models.inception_v3(pretrained=True)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\Pack/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

In [4]:
model_inception_extractor

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [11]:
for i in model_inception_extractor.parameters():
    i.requires_grad = False
    print(i.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [12]:
model_inception_extractor.AuxLogits.fc = nn.Linear(768, 2)
model_inception_extractor.fc = nn.Linear(2048, 2)

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [25]:
num_features = 25088
model_inception_extractor.classifier = nn.Linear(num_features, 2)
model_inception_extractor = model_inception_extractor.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_inception_extractor.parameters(), lr=0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

In [21]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=(15, 12))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)


def visualize_model(model, num_images=6):
    images_so_far = 0
    fig = plt.figure()

    for i, data in enumerate(dataloaders['val']):
        inp, lab = data
        inp, lab = inp.to(device), lab.to(device)

        outputs = model(inp)
        _, preds = torch.max(outputs.data, 1)

        for j in range(inp.size()[0]):
            images_so_far += 1
            imshow(inp.cpu().data[j], class_name[preds[j]])

            if images_so_far == num_images:
                return

In [22]:
def fit(model, criterion, optimizer, scheduler, num_epochs=25):
    losses = {'train': [], 'val': []}

    best_model_wts = model.state_dict()
    best_acc = 0.0

    pbar = trange(num_epochs, desc='Epoch:')
    for epoch in pbar:

        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for xb, yb in tqdm(dataloaders[phase], leave=True, desc=f'{phase} iter:'):
                xb, yb = xb.to(device), yb.to(device)

                if phase == 'train':
                    optimizer.zero_grad()

                if phase == 'eval':
                    with torch.no_grad():
                        outputs = model(xb)
                else:
                    outputs = model(xb)

                preds = torch.argmax(outputs, -1)
                loss = criterion(outputs, yb)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item()
                running_corrects += int(torch.sum(preds == yb.data))

            epoch_loss = running_loss / dataset_size[phase]
            epoch_acc = running_corrects / dataset_size[phase]

            losses[phase].append(epoch_loss)

            pbar.set_description(f'{phase} Loss: {epoch_loss}, Acc: {epoch_acc}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
    model.load_state_dict(best_model_wts)
    return model, losses

In [23]:
def evaluate(model):
    model.eval()

    running_correct = 0

    for xb, yb, in dataloaders['val']:
        xb, yb = xb.to(device), yb.to(device)

        outputs = model(xb)
        _, preds = torch.max(outputs, 1)

        running_correct += int(torch.sum(preds == yb))
    return running_correct / dataset_size['val']

In [ ]:
model_inception_extractor.aux_logits = False
model_inception_extractor, losses = fit(model_inception_extractor, loss_fn, optimizer, exp_lr_scheduler)

In [27]:
f'Accuracy: {evaluate(model_inception_extractor)}'

'Accuracy: 0.8823529411764706'

In [28]:
for param in model_inception_extractor.parameters():
    param.requires_grad = True

In [ ]:
model_inception_extractor.aux_logits = False
model_inception_extractor, losses = fit(model_inception_extractor, loss_fn, optimizer, exp_lr_scheduler)

In [30]:
f'Accuracy: {evaluate(model_inception_extractor)}'

'Accuracy: 0.9019607843137255'

In [31]:
torch.save(model_inception_extractor.state_dict(), 'InceptionNET.pth')
model_inception_extractor.load_state_dict(torch.load('InceptionNET.pth'))

<All keys matched successfully>

In [ ]:
visualize_model(model_inception_extractor)